# Getty Query Tutorial (Place of Death)
In this notebook, we perform the similar hunger-for-knowledge procedure for place of death in Wikidata and Getty Vocabulary. The main difference is that we generalize the query for single property to the property chain, this is, the path in kgtk.

In [1]:
import os
import json
import pandas as pd

from kgtk.functions import kgtk, kypher

## Step 0: Set up environment paths
Here we set needed Graph files and desired output files to environment paths.

In [3]:
# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

# Folder where database files store
data_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
os.environ['DATABASE'] = data_path
kgtk_environment_variables.append('DATABASE')

# Wikidata (all is much less than claims)
os.environ['WIKIDATA'] = data_path + "claims.tsv"
kgtk_environment_variables.append('WIKIDATA')

# Label file of Wikidata
os.environ['KGTK_LABEL_FILE'] = data_path + "labels.en.tsv"
kgtk_environment_variables.append('KGTK_LABEL_FILE')

# P31
os.environ['P31'] = data_path + "P31.tsv"
kgtk_environment_variables.append('P31')

# P279star
os.environ['P279STAR'] = data_path + "P279star.tsv"
kgtk_environment_variables.append('P279STAR')

# Folder of ULAN
ulan_path = data_path + "gvp/ULAN/"
os.environ['ULAN'] = ulan_path
kgtk_environment_variables.append('ULAN')

# Full file provided by Getty 
# Full file is much truncated in this process thus we will use concat files below instead.
# os.environ['ULAN_FULL'] = ulan_path + "full.tsv"
# kgtk_environment_variables.append('ULAN_FULL')

# File concatenated by used explicit files: ULAN Subjects, AgentMap, Biographies, TGN-Wikidata Align
# The suffix ID means we performed add-id operation to the graph to meet the requirement of paths command.
ulan_concat_path = ulan_path + "ulan.concat.id.tsv"
os.environ['ULAN_CONCAT_ID'] = ulan_concat_path
kgtk_environment_variables.append('ULAN_CONCAT_ID')

# ULAN-Wikidata alignment file, maps ULAN ID to Wikidata Qnode
ulan_wikialign_path = ulan_path + "wiki.align.tsv"
os.environ['ULAN_ALIGN'] = ulan_wikialign_path
kgtk_environment_variables.append('ULAN_ALIGN')

# Output
output_path = data_path + "gvp/pod_output/"
if not os.path.exists(output_path):
    os.mkdir(output_path)
os.environ['OUTPUT'] = output_path
kgtk_environment_variables.append('OUTPUT')

# Each file will be explained later in the procedure
output_names = {
    "wiki_pod": "wiki.pod.tsv",
    "wiki_unknown": "wiki.unknown.tsv",
    "pairs": "pairs.tsv",
    "paths": "paths.tsv",
    "paths_label": "paths.label.tsv",
    "getty_pod": "getty.pod.tsv",
    "getty_mapped": "getty.mapped.tsv",
    "getty_agree": "getty.agree.tsv",
    "correct": "correct.tsv",
    "unknown": "unknown.tsv"
}

for key, value in output_names.items():
    variable = key.upper()
    os.environ[variable] = os.path.join(output_path, value)
    kgtk_environment_variables.append(variable)

for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

DATABASE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
WIKIDATA: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/claims.tsv"
KGTK_LABEL_FILE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/labels.en.tsv"
P31: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P31.tsv"
P279STAR: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P279star.tsv"
ULAN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/"
ULAN_CONCAT_ID: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/ulan.concat.id.tsv"
ULAN_ALIGN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/wiki.align.tsv"
OUTPUT: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pod_output/"
WIKI_POD: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pod_output/wiki.pod.tsv"
WIKI_UNKNOWN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pod_output/wiki.unknown.tsv"
PAIRS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pod_output/pairs.tsv"
PATHS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/pod_o

## Step 1: Query Wikidata
Query Wikidata for PoD for people that have an ULAN ID (92k people)

In [3]:
# count total poeple
# ULAN ID
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_627_c1.""node1"")"
0,92353


In [4]:
# Qnode
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c1.""node2"")"
0,92210


In [ ]:
## %%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P20]->(pod)' 
        --return 'distinct ulanid as node1, "P20" as label, pod as node2'
        --limit 10
    / add-labels
    """)

In [6]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P20]->(pod)' 
        --return 'distinct ulanid as node1, "P20" as label, pod as node2'
        -o $WIKI_POD
    """)

CPU times: user 8.24 ms, sys: 12.5 ms, total: 20.7 ms
Wall time: 12.1 s


## Step 2: Record Known and Unknown

In [7]:
# count known ULAN IDs
kgtk("""
    query -i $WIKI_POD
        --match '(ulanid)-[]->(pod)' 
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_671_c1.""node1"")"
0,49851


In [4]:
# count known Qnodes
kgtk("""
    query -i $WIKI_POD $ULAN_ALIGN
        --match 'p: (ulanid)-[]->(nation), a: (ulanid)-[]->(qnode)' 
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c2.""node2"")"
0,49757


Filter unknow in ULAN-Wikidata alignment

In [8]:
%%time
kgtk("""
    ifnotexists -i $ULAN_ALIGN 
        --filter-on $WIKI_POD 
        --input-keys node1 
        --filter-keys node1 
        -o $WIKI_UNKNOWN
    """)

CPU times: user 4.71 ms, sys: 9.69 ms, total: 14.4 ms
Wall time: 4.45 s


In [9]:
# count unknown ULAN IDs
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_672_c1.""node1"")"
0,42502


In [5]:
# count unknown Qnodes
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_672_c1.""node2"")"
0,42453


## Step 3: Query Getty
Based on the known PoB for people, query Getty. Before we perform single property query (we can consider it as 1 hop paths query), this time, we generalize to paths between known pairs (Subjects, Objects). Similarily we will compute the distributions of found paths and pick the top 1 path (property chain). 

### 3-1: Build Pairs from Wikidata Results 
Pairs file is needed for performing kgtk paths operation as the query objects.

In [10]:
%%time
kgtk("""
    query -i $WIKI_POD
        --match '(ulanid)-[]->(pod)'
        --return 'distinct ulanid as source, pod as target'
        --limit 200
        -o $PAIRS
    """)

CPU times: user 6.85 ms, sys: 8.73 ms, total: 15.6 ms
Wall time: 4.76 s


In [11]:
!head $PAIRS | column -ts $'\t'

source          target
ulan:500023949  Q90
ulan:500123827  Q60
ulan:500072302  Q1781
ulan:500091345  Q1781
ulan:500020713  Q269174
ulan:500020713  Q3798
ulan:500020713  Q64
ulan:500160869  Q504255
ulan:500113074  Q1781


### 3-2: Perform Paths Mapping
This step is the core step of this query and it is also the bottleneck of running this procedure.

In [12]:
%%time
kgtk("""
    paths --max_hops 4
        --path-file $PAIRS
        --path-mode NONE 
        --path-source source
        --path-target target
        -i $ULAN_CONCAT_ID
        --statistics-only
        -o $PATHS
    """)

CPU times: user 134 ms, sys: 72.4 ms, total: 206 ms
Wall time: 3min 41s


In [13]:
!head $PATHS

node1	label	node2	id
p0	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p0-0-0
p0	1	ulan:500023949-agent:gvp:biographyPreferred:ulan_bio:4000061264	p0-1-1
p0	2	ulan_bio:4000061264:schema:birthPlace:tgn:7008038-place	p0-2-2
p0	3	tgn:7008038-place:skos:exactMatch:Q90	p0-3-3
p1	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p1-0-4
p1	1	ulan:500023949-agent:gvp:biographyNonPreferred:ulan_bio:4000174196	p1-1-5
p1	2	ulan_bio:4000174196:schema:deathPlace:tgn:7008038-place	p1-2-6
p1	3	tgn:7008038-place:skos:exactMatch:Q90	p1-3-7
p2	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p2-0-8


### 3-3: Select the Top 1 Property Chain
Here we use Pandas to sort the top 1 property chain. 

In [14]:
# load paths.tsv file
paths = pd.read_csv(os.environ['PATHS'], sep='\t')
# process property: since paths command generate id as node2, we need to further extract property from it
paths.node2 = paths.node2.apply(lambda x: ':'.join(x.split(':')[2:4]))
# save processed property
paths.to_csv(os.environ['PATHS_LABEL'], sep='\t', index=False)
# join property chain for each found path
paths_concat = paths.groupby(paths['node1']).agg({'node2': lambda x: ' '.join(list(x))})
# sort the top 
paths_concat.value_counts()#.head(1)

node2                                                                 
foaf:focus gvp:biographyPreferred schema:deathPlace skos:exactMatch       52
foaf:focus gvp:biographyPreferred schema:birthPlace skos:exactMatch       19
foaf:focus gvp:biographyNonPreferred schema:deathPlace skos:exactMatch     7
foaf:focus gvp:biographyNonPreferred schema:birthPlace skos:exactMatch     4
dtype: int64

### 3-4: Query Getty Using the Property Chain
Query Getty for the values of the unknown people, with the found property chain: `(ulanid)-[foaf:focus]->()-[gvp:biographyPreferred]->()-[schema:deathPlace]->()-[skos:exactMatch]->(Qnode of PoD)`

In [15]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(pod)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:deathPlace" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P20" as label, pod as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 40.2 ms, sys: 25 ms, total: 65.2 ms
Wall time: 1min 25s


,node1,label,node2,label;label,node2;label
0,ulan:500000011,P20,Q84,'place of death'@en,'London'@en
1,ulan:500000037,P20,Q90,'place of death'@en,'Paris'@en
2,ulan:500000057,P20,Q1164,'place of death'@en,'Assam'@en
3,ulan:500000092,P20,Q62,'place of death'@en,'San Francisco'@en
4,ulan:500000114,P20,Q2044,'place of death'@en,'Florence'@en
5,ulan:500000159,P20,Q16554,'place of death'@en,'Denver'@en
6,ulan:500000164,P20,Q40435,'place of death'@en,'Buffalo'@en
7,ulan:500000320,P20,Q90,'place of death'@en,'Paris'@en
8,ulan:500000387,P20,Q84,'place of death'@en,'London'@en
9,ulan:500000424,P20,Q1384,'place of death'@en,'New York'@en


In [16]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(pod)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:deathPlace" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P20" as label, pod as node2'
        -o $GETTY_POD
    """)

CPU times: user 10.3 ms, sys: 13.7 ms, total: 23.9 ms
Wall time: 17.2 s


Count how many new results (count in ULAN ID) we found

In [17]:
kgtk("""
    query -i $GETTY_POD
        --match '(ulanid)-[]->(pod)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_673_c1.""node1"")"
0,2554


## Step 4: Record New Results

Map back to Wikidata

In [18]:
%%time
kgtk("""
    query -i $GETTY_POD $ULAN_ALIGN
        --match 'g: (ulanid)-[p]->(pod), 
                 w: (ulanid)-[]->(qnode)'
        --return 'distinct qnode as node1, p.label as label, pod as node2'
        -o $GETTY_MAPPED
    """)

CPU times: user 8.34 ms, sys: 8.72 ms, total: 17.1 ms
Wall time: 4.86 s


Count how many new results (count in Qnode) we found

In [19]:
kgtk("""
    query -i $GETTY_MAPPED
        --match '(qnode)-[]->(pod)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_674_c1.""node1"")"
0,2556


## Step 5: Validate with Wikidata Constraints

Count how many new results we found (Qnodes)

In [21]:
%%time
kgtk("""
    query -i $GETTY_MAPPED $P31 $P279STAR \
        --match 'g: (node1)-[nodeProp]->(node2), P31: (node2)-[]->(nodex), P279star: (nodex)-[]->(par)' \
        --where 'par in ["Q3895768", "Q24334893", "Q2221906", "Q102496", "Q27096213", "Q16917", "Q2945655", "Q3591565", "Q837142", "Q608152", "Q2385804"]' \
        --return 'distinct node1 as `node1`, nodeProp.label as `label`, node2 as `node2`' \
        -o $CORRECT
    """)

CPU times: user 7.73 ms, sys: 8.32 ms, total: 16.1 ms
Wall time: 4.41 s


In [22]:
%%time
kgtk("""
    ifnotexists -i $GETTY_MAPPED \
        --filter-on $CORRECT \
        --input-keys node1 node2 \
        --filter-keys node1 node2
    """)

CPU times: user 10.6 ms, sys: 14.2 ms, total: 24.8 ms
Wall time: 4.39 s


,node1,label,node2


## Step 6: Record Unknown after Query
We record unknowns after query Getty.

In [23]:
kgtk("""
    ifnotexists -i $WIKI_UNKNOWN \
        --filter-on $GETTY_POD \
        --input-keys node1 \
        --filter-keys node1
        -o $UNKNOWN
    """)

In [24]:
# count in ULAN IDs
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_675_c1.""node1"")"
0,39948


In [6]:
# count in Qnodes
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_675_c1.""node2"")"
0,39906


# Step 7: Comparison of the Known Values
How often do the results from two knowledge graphs agree

In [4]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $ULAN_CONCAT_ID
        --match 'w: (ulanid)-[]->(), 
                 u: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(pod)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:deathPlace" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P20" as label, pod as node2'
        -o $GETTY_AGREE
    """)

CPU times: user 9.97 ms, sys: 17.2 ms, total: 27.1 ms
Wall time: 8.87 s


Count agree in Qnode:

In [5]:
%%time
kgtk("""
    query -i $WIKI_POD $GETTY_AGREE $ULAN_ALIGN
        --match 'd: (ulanid)-[]->(pod), g: (ulanid)-[]->(pod), a: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

CPU times: user 9.71 ms, sys: 11.2 ms, total: 20.9 ms
Wall time: 4.65 s


,"count(DISTINCT graph_627_c3.""node2"")"
0,12597


Count value per node:

In [6]:
nodes = kgtk("""
    query -i $GETTY_AGREE $ULAN_ALIGN 
        --match 'g: (ulanid)-[]->(), w: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode) as node'
    """)

In [7]:
import subprocess

values = subprocess.check_output("wc -l < $GETTY_AGREE", shell=True)
values = values.decode("utf-8").strip()
values = int(values) - 1

In [8]:
values / nodes.iloc[0]['node']

1.0000585069038146